In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EvalPrediction,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from tqdm import tqdm
import json
import wandb
from datetime import datetime

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Configuration
class Config:
    # Paths
    data_dir = "/content/drive/MyDrive/preprocessed_output"
    output_dir = "/content/drive/MyDrive/authorship_models"

    # Model selection (choose one: 'bpe', 'wordpiece', 'sentencepiece')
    tokenization_method = 'sentencepiece'  # Change this to switch between methods

    # Model configurations
    model_configs = {
        'bpe': {
            'model_name': 'xlm-roberta-base',
            'train_data': 'bpe_train_dataset.pt',
            'val_data': 'bpe_val_dataset.pt',
            'test_data': 'bpe_test_dataset.pt'
        },
        'wordpiece': {
            'model_name': 'bert-base-multilingual-cased',
            'train_data': 'wordpiece_train_dataset.pt',
            'val_data': 'wordpiece_val_dataset.pt',
            'test_data': 'wordpiece_test_dataset.pt'
        },
        'sentencepiece': {
            'model_name': 'xlm-roberta-large',
            'train_data': 'sentencepiece_train_dataset.pt',
            'val_data': 'sentencepiece_val_dataset.pt',
            'test_data': 'sentencepiece_test_dataset.pt'
        }
    }

    # Training hyperparameters
    learning_rate = 5e-5
    batch_size = 8
    num_epochs = 20
    warmup_steps = 500
    weight_decay = 0.01
    max_length = 128

    # Training settings
    gradient_accumulation_steps = 4
    eval_steps = 100
    save_steps = 200
    logging_steps = 50

    # Early stopping
    early_stopping_patience = 3

    # Experiment tracking
    use_wandb = True
    project_name = "twitter-authorship-attribution"

config = Config()

# Create output directory
os.makedirs(config.output_dir, exist_ok=True)

# Initialize wandb if enabled
if config.use_wandb:
    wandb.init(
        project=config.project_name,
        name=f"{config.tokenization_method}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        config={
            "tokenization_method": config.tokenization_method,
            "model_name": config.model_configs[config.tokenization_method]['model_name'],
            "learning_rate": config.learning_rate,
            "batch_size": config.batch_size,
            "num_epochs": config.num_epochs
        }
    )

# Custom Dataset Class
class AuthorshipDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Load datasets
print(f"Loading {config.tokenization_method} datasets...")
model_config = config.model_configs[config.tokenization_method]

train_data = torch.load(os.path.join(config.data_dir, model_config['train_data']))
val_data = torch.load(os.path.join(config.data_dir, model_config['val_data']))
test_data = torch.load(os.path.join(config.data_dir, model_config['test_data']))

# Get number of authors (classes)
num_authors = len(set(train_data['labels'].tolist()))
print(f"Number of authors: {num_authors}")

# Create datasets
train_dataset = AuthorshipDataset(
    train_data['input_ids'],
    train_data['attention_mask'],
    train_data['labels']
)

val_dataset = AuthorshipDataset(
    val_data['input_ids'],
    val_data['attention_mask'],
    val_data['labels']
)

test_dataset = AuthorshipDataset(
    test_data['input_ids'],
    test_data['attention_mask'],
    test_data['labels']
)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Load model and tokenizer
print(f"Loading model: {model_config['model_name']}")
model = AutoModelForSequenceClassification.from_pretrained(
    model_config['model_name'],
    num_labels=num_authors,
    ignore_mismatched_sizes=True
)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_config['model_name'])

# Define metrics
def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    # Get predicted classes
    preds = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )

    # Top-5 accuracy
    top_5_preds = np.argsort(predictions, axis=1)[:, -5:]
    top_5_acc = np.mean([label in top_5_preds[i] for i, label in enumerate(labels)])

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top_5_accuracy': top_5_acc
    }

    return metrics

# Define training arguments
training_args = TrainingArguments(
    output_dir=os.path.join(config.output_dir, f"{config.tokenization_method}_model"),
    eval_strategy="steps",  # Changed from evaluation_strategy
    eval_steps=config.eval_steps,
    save_strategy="no",  # Don't save during training
    # save_steps=config.save_steps,  # Comment this out
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.num_epochs,
    weight_decay=config.weight_decay,
    warmup_steps=config.warmup_steps,
    logging_dir=os.path.join(config.output_dir, "logs"),
    logging_steps=config.logging_steps,
    load_best_model_at_end=False,  # Changed to False since we're not saving checkpoints
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    fp16=True,  # Enable mixed precision training
    report_to="wandb" if config.use_wandb else "none",
    run_name=f"{config.tokenization_method}_authorship",
    # save_total_limit=3,  # Comment this out
    push_to_hub=False,
    do_train=True,  # Added
    do_eval=True,   # Added
)
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=config.early_stopping_patience)]
)

# Train the model
print("\nStarting training...")
train_result = trainer.train()

# Save the final model
trainer.save_model()
trainer.save_state()

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = trainer.evaluate(eval_dataset=test_dataset)

# Save results
results = {
    'tokenization_method': config.tokenization_method,
    'model_name': model_config['model_name'],
    'train_results': train_result.metrics,
    'test_results': test_results,
    'num_authors': num_authors,
    'training_samples': len(train_dataset),
    'validation_samples': len(val_dataset),
    'test_samples': len(test_dataset)
}

with open(os.path.join(config.output_dir, f"{config.tokenization_method}_results.json"), 'w') as f:
    json.dump(results, f, indent=2)

# Generate detailed predictions for error analysis
print("\nGenerating detailed predictions...")
test_predictions = trainer.predict(test_dataset)
predictions = np.argmax(test_predictions.predictions, axis=1)
true_labels = test_predictions.label_ids

# Calculate per-author metrics
from collections import defaultdict
author_metrics = defaultdict(lambda: {'correct': 0, 'total': 0})

for pred, true in zip(predictions, true_labels):
    author_metrics[true]['total'] += 1
    if pred == true:
        author_metrics[true]['correct'] += 1

# Calculate per-author accuracy
author_accuracies = {
    author: metrics['correct'] / metrics['total']
    for author, metrics in author_metrics.items()
}

# Plot confusion matrix (for a subset of authors if too many)
num_authors_to_plot = min(20, num_authors)
if num_authors > num_authors_to_plot:
    # Select top authors with most errors
    author_errors = [(author, 1 - acc) for author, acc in author_accuracies.items()]
    author_errors.sort(key=lambda x: x[1], reverse=True)
    authors_to_plot = [x[0] for x in author_errors[:num_authors_to_plot]]

    # Filter predictions and labels for selected authors
    mask = np.isin(true_labels, authors_to_plot)
    filtered_predictions = predictions[mask]
    filtered_labels = true_labels[mask]

    # Create mapping for plotting
    author_mapping = {author: i for i, author in enumerate(authors_to_plot)}
    filtered_predictions_mapped = [author_mapping.get(p, -1) for p in filtered_predictions]
    filtered_labels_mapped = [author_mapping.get(l, -1) for l in filtered_labels]
else:
    filtered_predictions_mapped = predictions
    filtered_labels_mapped = true_labels
    authors_to_plot = list(range(num_authors))

# Create confusion matrix
cm = confusion_matrix(filtered_labels_mapped, filtered_predictions_mapped)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title(f'Confusion Matrix - {config.tokenization_method.upper()} Model')
plt.xlabel('Predicted Author')
plt.ylabel('True Author')
plt.tight_layout()
plt.savefig(os.path.join(config.output_dir, f'{config.tokenization_method}_confusion_matrix.png'))
plt.close()

# Plot per-author accuracy
plt.figure(figsize=(15, 8))
sorted_authors = sorted(author_accuracies.items(), key=lambda x: x[1])
authors, accuracies = zip(*sorted_authors[:30])  # Show worst 30 authors
plt.bar(range(len(authors)), accuracies)
plt.xlabel('Author ID')
plt.ylabel('Accuracy')
plt.title(f'Per-Author Accuracy (Worst 30) - {config.tokenization_method.upper()}')
plt.xticks(range(len(authors)), authors, rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(config.output_dir, f'{config.tokenization_method}_per_author_accuracy.png'))
plt.close()

# Advanced analysis functions
def analyze_errors(predictions, true_labels, test_data, author_map):
    """Analyze prediction errors in detail"""
    errors = []

    for i, (pred, true) in enumerate(zip(predictions, true_labels)):
        if pred != true:
            errors.append({
                'index': i,
                'true_author': true,
                'predicted_author': pred,
                'text_length': test_data['input_ids'][i].ne(0).sum().item()
            })

    return errors

def get_most_confused_pairs(cm, n=10):
    """Find the most confused author pairs"""
    confused_pairs = []

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i != j and cm[i, j] > 0:
                confused_pairs.append((i, j, cm[i, j]))

    confused_pairs.sort(key=lambda x: x[2], reverse=True)
    return confused_pairs[:n]

# Perform error analysis
errors = analyze_errors(predictions, true_labels, test_data, train_data['author_map'])
error_rate = len(errors) / len(true_labels)
print(f"\nError rate: {error_rate:.2%}")

# Find most confused author pairs
confused_pairs = get_most_confused_pairs(cm)
print("\nMost confused author pairs:")
for true_author, pred_author, count in confused_pairs:
    print(f"True: {true_author}, Predicted: {pred_author}, Count: {count}")

# Save error analysis - Fixed version
error_analysis = {
    'error_rate': float(error_rate),  # Convert to float
    'total_errors': int(len(errors)),  # Convert to int
    'most_confused_pairs': [(int(t), int(p), int(c)) for t, p, c in confused_pairs],  # Convert all to int
    'per_author_accuracy': {str(k): float(v) for k, v in author_accuracies.items()}  # Convert values to float
}

with open(os.path.join(config.output_dir, f'{config.tokenization_method}_error_analysis.json'), 'w') as f:
    json.dump(error_analysis, f, indent=2)

# Print final summary
print("\n" + "="*50)
print(f"TRAINING COMPLETE - {config.tokenization_method.upper()}")
print("="*50)
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test F1 Score: {test_results['eval_f1']:.4f}")
print(f"Test Top-5 Accuracy: {test_results['eval_top_5_accuracy']:.4f}")
print(f"Model saved to: {os.path.join(config.output_dir, f'{config.tokenization_method}_model')}")
print("="*50)


Mounted at /content/drive
Using device: cuda


Loading sentencepiece datasets...
Number of authors: 100
Train samples: 14000
Validation samples: 3000
Test samples: 3000
Loading model: xlm-roberta-large


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Top 5 Accuracy
100,4.631600,4.537568,0.034000,0.020192,0.034940,0.034000,0.105667
200,4.608800,4.539815,0.018000,0.007164,0.007756,0.018000,0.082667
300,4.222200,4.018181,0.124333,0.090170,0.117506,0.124333,0.258333
400,3.439700,3.124622,0.272000,0.223616,0.252828,0.272000,0.467333
500,2.857700,2.708831,0.349333,0.299123,0.323369,0.349333,0.562667
600,2.580300,2.449840,0.396000,0.362556,0.405057,0.396000,0.644667
700,2.469000,2.293708,0.422000,0.389990,0.431959,0.422000,0.672667
800,2.324300,2.165477,0.445000,0.423245,0.480123,0.445000,0.708333
900,2.108100,2.051596,0.459333,0.439065,0.515031,0.459333,0.738000
1000,1.967600,1.932677,0.493000,0.472177,0.523319,0.493000,0.753333


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


Evaluating on test set...



Generating detailed predictions...

Error rate: 36.67%

Most confused author pairs:
True: 2, Predicted: 0, Count: 23
True: 5, Predicted: 0, Count: 22
True: 1, Predicted: 0, Count: 20
True: 17, Predicted: 0, Count: 19
True: 3, Predicted: 0, Count: 18
True: 4, Predicted: 0, Count: 17
True: 6, Predicted: 0, Count: 17
True: 8, Predicted: 0, Count: 16
True: 9, Predicted: 0, Count: 16
True: 10, Predicted: 0, Count: 15

Creating comparison script...

TRAINING COMPLETE - SENTENCEPIECE
Test Accuracy: 0.6333
Test F1 Score: 0.6345
Test Top-5 Accuracy: 0.8263
Model saved to: /content/drive/MyDrive/authorship_models/sentencepiece_model
